## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim


In [111]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [112]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 50 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [113]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)


In [114]:
params.kpi = 3

### Profit Mazimization - Begin 

In [115]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

18-07-23 13:04:26-INFO-Initializing pairwise trip shareability between 50 and 50 trips.
18-07-23 13:04:26-INFO-creating combinations
18-07-23 13:04:26-INFO-2450	 nR*(nR-1)
18-07-23 13:04:26-INFO-Reduction of feasible pairs by 99.35%
18-07-23 13:04:26-INFO-Degree 2 	Completed
18-07-23 13:04:26-INFO-trips to extend at degree 2 : 42
18-07-23 13:04:27-INFO-At degree 2 feasible extensions found out of 0 searched
18-07-23 13:04:27-INFO-Degree 3 	Completed
18-07-23 13:04:27-INFO-No more trips to exted at degree 3


In [116]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

18-07-23 13:04:27-INFO-Matching 50 trips to 92 rides in order to min u_veh
18-07-23 13:04:27-INFO-Problem solution: Optimal. 
Total costs for single trips:         13,338 
reduced by matching to:               11,893


In [117]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],4.147000,224,1,[4.147],"[0, 224]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44869226, 4244384221]","[None, 0, 0]",node time req_id od 0 ...,224,2.24,3.363000,0.672600,2.6904
1,[1],5.283000,285,1,[5.2829999999999995],"[114, 285]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44860729, 44830128]","[None, 1, 1]",node time req_id od 0 NaN...,285,2.85,4.285500,0.857100,3.4284
2,[2],0.863000,46,1,[0.863],"[122, 46]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44773458, 626091907]","[None, 2, 2]",node time req_id od 0 N...,46,0.46,0.702000,0.140400,0.5616
3,[3],5.292500,286,1,[5.2925],"[253, 286]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1519889798, 44778405]","[None, 3, 3]",node time req_id od 0 ...,286,2.86,4.291500,0.858300,3.4332
4,[4],3.225000,174,1,[3.225],"[324, 174]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44842366, 1413911044]","[None, 4, 4]",node time req_id od 0 ...,174,1.74,2.616000,0.523200,2.0928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,"[10, 12]",6.079900,360,21,"[4.9136, 1.1663000000000001]","[945.5, 69, 69, 222]","[10, 12]","[12, 10]",2,87,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1620328022, 44848878, 4862057241, 16797...","[None, 10, 12, 12, 10]",node time req_id od 0 ...,360,3.60,3.996000,0.799200,3.1968
88,"[34, 38]",14.611750,601,21,"[7.889900000000001, 6.72185]","[2430.5, 134, 415, 52]","[34, 38]","[38, 34]",2,88,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 2298434570, 1413911113, 1402734577, 370...","[None, 34, 38, 38, 34]",node time req_id od 0 ...,601,6.01,9.675000,1.935000,7.7400
89,"[47, 49]",12.013800,486,21,"[6.246475, 5.7673250000000005]","[3367.0, 92, 353, 41]","[47, 49]","[49, 47]",2,89,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 5008459646, 44814123, 4555788280, 43007...","[None, 47, 49, 49, 47]",node time req_id od 0 ...,486,4.86,7.823250,1.564650,6.2586
90,"[17, 21]",13.929525,690,21,"[9.684325000000001, 4.2452000000000005]","[1204.5, 372, 245, 73]","[17, 21]","[21, 17]",2,90,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1448535745, 44831869, 5353328924, 49865...","[None, 17, 21, 21, 17]",node time req_id od 0 ...,690,6.90,8.933625,1.786725,7.1469


In [119]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

0.0005
18-07-23 13:05:02-CRITICAL-this is request 0 with [0, 71] available rides.
18-07-23 13:05:02-CRITICAL-ride 0 available [1]
18-07-23 13:05:02-CRITICAL-ride 71 available [1, 0]
18-07-23 13:05:02-CRITICAL-this is reuqest 0 with [0, 71] still available rides.
hell
18-07-23 13:05:02-CRITICAL-vehicle 1 has 2 choices
0.0005
18-07-23 13:05:02-CRITICAL-this is request 1 with [1, 57, 71] available rides.
18-07-23 13:05:02-CRITICAL-ride 1 available [1]
18-07-23 13:05:02-CRITICAL-ride 57 available [1, 0]
18-07-23 13:05:02-CRITICAL-ride 71 not available [5, 1]
18-07-23 13:05:02-CRITICAL-this is reuqest 1 with [1, 57] still available rides.
hell
18-07-23 13:05:02-CRITICAL-vehicle 16 has 2 choices
0.0005
18-07-23 13:05:02-CRITICAL-this is request 2 with [2] available rides.
18-07-23 13:05:02-CRITICAL-ride 2 available [1]
18-07-23 13:05:02-CRITICAL-this is reuqest 2 with [2] still available rides.
hell
18-07-23 13:05:02-CRITICAL-vehicle 5 has 1 choices
0.0005
18-07-23 13:05:02-CRITICAL-this is 

In [120]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,STARTS_REPOSITIONING,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM
1,5.0,0.0,False,0,0,281.0,45.0,0,243.0,0,...,0,0,0,8825.0,807.0,0,448.0,438.0,7132.0,12.793
2,3.0,0.0,False,0,0,1926.0,30.0,0,161.0,0,...,0,0,0,7645.0,830.0,0,117.0,281.0,6417.0,12.137
3,1.0,0.0,False,0,0,324.0,15.0,0,20.0,0,...,0,0,0,10193.0,174.0,0,43.0,65.0,9911.0,1.744
4,2.0,0.0,False,0,0,2264.0,15.0,0,261.0,0,...,0,0,0,7822.0,148.0,0,229.0,336.0,7109.0,5.736
5,3.0,0.0,False,0,0,2200.0,30.0,0,68.0,0,...,0,0,0,7936.0,261.0,0,214.0,188.0,7273.0,4.015
6,5.0,0.0,False,0,0,1864.0,45.0,0,255.0,0,...,0,0,0,7248.0,950.0,0,287.0,450.0,5561.0,15.389
7,1.0,0.0,False,0,0,2146.0,15.0,0,20.0,0,...,0,0,0,8175.0,262.0,0,151.0,65.0,7697.0,2.627
8,2.0,0.0,False,0,0,3326.0,15.0,0,100.0,0,...,0,0,0,6736.0,373.0,0,189.0,175.0,5999.0,6.954
9,3.0,0.0,False,0,0,662.0,45.0,0,60.0,0,...,0,0,0,9079.0,488.0,0,375.0,195.0,8021.0,4.905
10,5.0,0.0,False,0,0,1368.0,45.0,0,283.0,0,...,0,0,0,7296.0,1022.0,0,635.0,478.0,5161.0,17.316


In [121]:
sim.vehs[1].rdf

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,sim_schedule,ttrav,dist,fare,commission,driver_revenue,pickup_dist,trav_dist,operating_cost,profit
71,"[0, 1]",8.293475,396,20,"[3.7826000000000004, 4.510875]","[1.0, 112, 164, 120]","[0, 1]","[0, 1]",2,71,...,node time req_id od 0 ...,396,3.96,5.736375,1.147275,4.5891,2248,2251.96,1.125980,3.463120
64,"[7, 15]",10.884400,706,20,"[5.19205, 5.69235]","[750.0, 328, 79, 299]","[7, 15]","[7, 15]",2,64,...,node time req_id od 0 ...,706,7.06,7.094250,1.418850,5.6754,746,753.06,0.376530,5.298870
51,"[22, 27]",9.638600,589,20,"[3.3958500000000003, 6.24275]","[1690.5, 134, 83, 372]","[22, 27]","[22, 27]",2,51,...,node time req_id od 0 ...,589,5.89,6.194250,1.238850,4.9554,859,864.89,0.432445,4.522955


In [122]:
pd.DataFrame(sim.vehs[1].myrides)

,veh,pos,t,event,paxes
0,1,4.484406e+07,0,STARTS_DAY,[]
1,1,4.484406e+07,0,OPENS_APP,[]
2,1,4.484406e+07,0,RECEIVES_REQUEST,[]
3,1,4.484406e+07,15,ACCEPTS_REQUEST,[]
4,1,4.484406e+07,35,IS_ACCEPTED_BY_TRAVELLER,[None]
5,1,4.484406e+07,149,IS_ACCEPTED_BY_TRAVELLER,[None]
6,1,4.486923e+07,252,ARRIVES_AT_PICKUP,[]
7,1,4.486923e+07,252,MEETS_TRAVELLER_AT_PICKUP,[]
8,1,4.486923e+07,282,DEPARTS_FROM_PICKUP,[0]
9,1,4.486073e+07,394,ARRIVES_AT_PICKUP,[0]


In [85]:
responses = []
avg_kpi = []
idle_time = []

for i in range(2, 3):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

0.0005
18-07-23 12:53:21-CRITICAL-this is request 0 with [0] available rides.
18-07-23 12:53:21-CRITICAL-ride 0 available [1]
18-07-23 12:53:21-CRITICAL-this is reuqest 0 with [0] still available rides.
18-07-23 12:53:21-CRITICAL-vehicle 1 has 1 choices
0.0005
18-07-23 12:53:21-CRITICAL-this is request 1 with [1] available rides.
18-07-23 12:53:21-CRITICAL-ride 1 available [1]
18-07-23 12:53:21-CRITICAL-this is reuqest 1 with [1] still available rides.
18-07-23 12:53:21-CRITICAL-vehicle 14 has 1 choices
0.0005
18-07-23 12:53:21-CRITICAL-this is request 2 with [2] available rides.
18-07-23 12:53:21-CRITICAL-ride 2 available [1]
18-07-23 12:53:21-CRITICAL-this is reuqest 2 with [2] still available rides.
18-07-23 12:53:21-CRITICAL-vehicle 17 has 1 choices
0.0005
18-07-23 12:53:21-CRITICAL-this is request 3 with [3] available rides.
18-07-23 12:53:21-CRITICAL-ride 3 available [1]
18-07-23 12:53:21-CRITICAL-this is reuqest 3 with [3] still available rides.
18-07-23 12:53:21-CRITICAL-vehicl